In [1]:
#Imports
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

In [2]:
#Paths & model
main_folder = "C:/Users/Rostom/Desktop/Misc/Git Projects/Projet-Integrateur-Classification-d-images/CNN_model_train/archive/"
IMGS_ATTR_PATH = main_folder+'list_attr_celeba.csv'
IMGS_PATH = main_folder+'img_align_celeba/img_align_celeba'
FACE_WEIGHT_VGG16 = './vgg16.h5'
MODEL_20_EPOCHS = './model_20_epochs.h5'


In [3]:
#Function
def display_image(image,result,features):
    s=""
    for i, label in enumerate(features.columns):
        pred = result[0][i]
        s += f"{label}: predicted {1 if pred > 0.4 else 0} ({format(pred, '.4f')})"
        s +="\n"
    plt.imshow(image)
    plt.title(s)
    plt.figure()
    return s

def load_model(loss, optimizer='adam', metrics=['accuracy']):
    resnet_features = ResNet50(include_top=False, input_shape=(112, 112, 3), pooling='avg')
    for layer in resnet_features.layers:
        layer.trainable = False
    model = Sequential([resnet_features,Dense(len(attr2idx)),Activation('sigmoid') ])
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)  
    model = keras.models.load_model(glob.glob('./model_20_epochs.h5')[0])
    return model

def get_attr_names(dataframe):
    attr_names = []
    attr_names = list(dataframe.columns)
    # removing images_id column
    del attr_names[0]
    return attr_names

In [4]:
#Variables
attr_df = pd.read_csv(IMGS_ATTR_PATH)
attr_names = get_attr_names(attr_df)
attr2idx = {v:i for i, v in enumerate(attr_names)}
attr_df = pd.read_csv(IMGS_ATTR_PATH)

In [5]:
#Creating model
optimizer = 'adam'
bc_loss = 'binary_crossentropy'
metrics = ['binary_accuracy']
model = load_model(bc_loss, optimizer, metrics)

NameError: name 'glob' is not defined

In [ ]:
#Loading image
image = tf.io.read_file('./image_samples/321240135_698995238546479_983226794941031556_n.jpg')
image = tf.io.decode_jpeg(contents=image, channels=3)
image = tf.cast(image, tf.float32)
image = tf.divide(image, 255.)
image = tf.image.resize(image, size=(112, 112))
# image.shape
# plt.imshow(image)

In [ ]:
#Prediction
new_image = image[None, :, :, :]
result = model.predict(new_image)
features = attr_df.drop(columns=['image_id'])
display_image(new_image[0],result,features)